**Case: Optimering af Online Retail Platform**

Du arbejder som en dataingeniør i en online detailhandelsvirksomhed ved navn "X," der ønsker at forbedre sin kundeservice og salgsstrategi ved at udnytte Big Data. Virksomheden har adgang til en bred vifte af datakilder, herunder transaktionsdata, brugeradfærd på webstedet, sociale mediedata og eksterne markedsdata. Din opgave er at designe og opbygge en mindre Big Data løsning for X.


1. **Dataindsamling og Opsamling:**
   - Design en løsning til at indsamle og opsamle data fra forskellige kilder i realtid, inklusive transaktionsdata, webadfærd og sociale mediedata. Overvej anvendelse af HTTPS (REST), Kafka eller MQTT til dataindsamling.
   - Sikkerhedsaspekter er vigtige. Implementer kryptering af data under transport for at beskytte følsomme oplysninger.

In [2]:
#insert storageDrive.csv to Microsoft Sql Server database

import csv
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import urllib

df = pd.read_csv('walmart.csv')
#Print available drivers
#print(pyodbc.drivers())

import urllib

params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=tcp:nextchsolutions.database.windows.net,1433;DATABASE=NexTechSolutions;UID=ktfs;PWD=Password1!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=60;")
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params,fast_executemany=True)

In [3]:
#Put the 'raw data' into a SQL database
df.to_sql('WalmartDb', engine, if_exists='append', index=False)

-1

2. **Dataforberedelse og Transformation:**
   - Opbyg en pipeline til at rengøre, strukturere og forberede data til analyse. Dette inkluderer valg af filformater som CSV, XML, HDF5 osv.
   - Håndter forskellige datatyper, herunder struktureret og ustruktureret data.

In [12]:
import luigi
import pyodbc
import pandas as pd
import csv
import luigi

class ExtractDataTask(luigi.Task):
    def run(self):
        # Set up the connection to the SQL Server database
        conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=tcp:nextchsolutions.database.windows.net,1433;DATABASE=NexTechSolutions;UID=ktfs;PWD=Password1!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=60;'
        conn = pyodbc.connect(conn_str)
        print("Extracting the data..")
        # Define the SQL query
        query = '''
            SELECT * FROM WalmartDb
        '''

        # Execute the query and fetch the results into a pandas DataFrame
        df = pd.read_sql(query, conn)
        df.to_csv('walmart_extracted.csv', index=False)
        # Close the database connection
        conn.close()

    def output(self):
        return luigi.LocalTarget('walmart_extracted.csv')


class TransformDataTask(luigi.Task):
    def requires(self):
        return ExtractDataTask()

    def run(self):
        input_file = self.input().path
        df = pd.read_csv(input_file)
        df = df.drop_duplicates()
        df = df.dropna()
        df.to_csv('walmart_transformed.csv', index=False)

    def output(self):
        return luigi.LocalTarget('walmart_transformed.csv')


class LoadDataTask(luigi.Task):
    def requires(self):
        return TransformDataTask()

    def run(self):
        input_file = self.input().path
        with open(input_file, 'r') as file:
            data = [row for row in csv.reader(file)]

        with open('walmart_clean.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(data)

    def output(self):
        return luigi.LocalTarget('walmart_clean.csv')


class MyPipeline(luigi.WrapperTask):
    def requires(self):
        return LoadDataTask()


# if __name__ == '__main__':
luigi.run(main_task_cls=MyPipeline, local_scheduler=True)


usage: ipykernel_launcher.py [--local-scheduler] [--module CORE_MODULE]
                             [--help] [--help-all]
                             [Required root task]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"be2f6099-0975-4e01-9839-15e394fa0667" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\ssc\AppData\Roaming\jupyter\runtime\kernel-v2-25400Nj9z2BFN3RsJ.json


SystemExit: 2

c:\Users\ssc\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


3. **Analyse og Præsentation:**
   - Anvend dataanalyseværktøjer og teknikker til at trække indsigt fra de forberedte data.
   - Design dashboards eller rapporter for at give virksomhedens beslutningstagere nem adgang til relevante oplysninger.

4. **Sikkerhed og Overholdelse:**
   - Implementer omfattende sikkerhedsforanstaltninger for at beskytte data mod uautoriseret adgang og misbrug.
   - Overhold databeskyttelsesregler, især GDPR, når du håndterer brugerdata.

5. **Skalering og Optimering:**
   - Overvej hvordan løsningen kan skaleres i fremtiden med virksomhedens vækst og øgede datamængder.
   - Identificer muligheder for ydeevneoptimering.

Din Big Data løsning skal hjælpe RetailX med at træffe informerede beslutninger, forbedre kundeservicen og optimere salgsstrategien ved at udnytte de enorme datamængder, der er tilgængelige. Din evne til at integrere dataindsamling, forberedelse, analyse og sikkerhed vil være afgørende for succes i denne opgave.